In [2]:
import numpy as np
import time
import matplotlib.pyplot as plt

Question1 

To generate a random 2 d array from normal distribution numpy has a function numpy.random.randn
uses mean = 0 and standard deviation 1

Reference : https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html

Concept : 
In the context of linear regression, correlations refer to the degree to which independent variables are related to each other. Each column of random matrix is a feature independenlty sampled from normal distribution, by changing S we determine the degree of relation or correlations between these features 
S = I implies no correlations
Source : chat-gpt

coding structure -> self

In [3]:
def generate_input_data(n, S):
    m = 5 ## arbitary chosen no. of columns/features
    ##generate random matrix of size NXM
    random_matrix = np.random.randn(n,m)
    ##GENERATE INPUT DATA MATRIX using np.dot for matrix multiplication
    X = np.dot(random_matrix,S)
    ## X HAS no. of columns equal to desired outputs and each rows corresponds to sample
    return X
   
    

In [4]:
## test function
N = 100 ## SAMPLING SIZE
D = 2 # NO OF OUTPUTS
M = 5 # NO. OF FEATURES
S = np.eye(M,D)
#print(generate_input_data(N,S))

Q2 To add zero mean gaussian noise I used numpy.random.normal as it allows specifying sigma and mean unlike randn which uses default values ref: https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html 
dimensional mismatch (input N X D and noise is D+1 X 1) tacked by excluding 1st element from noise matrix
Concept : simulate real world scenarios target set itself not ideal maybe measurement noise or other gaussian noice to be added

In [5]:
def generate_target_matrix(X, w, sigma):
    ##check dimenstional mismatch   X = N X D while w = row vector of size D+1
    is_same = X.shape[1] == w.shape[0]
    if(not is_same):
        ## if mismatch reduce exclude 1st element
        w = w[1 :]  # excluded 0TH
        bias = w[0]
    else:
        bias = 0
    ##weighted input
    X_wt = np.dot(X,w)
    ##ADD BIAS
    X_biased = X_wt + bias
    ##add gaussian noise of variance sigma
    noise = np.random.normal(loc=0.0, scale=sigma, size= X_biased.shape)
    ##final target set
    t = X_biased + noise
    return t
    
    
    
    

In [6]:
## test target set generation
X = generate_input_data(N,S)
sigma = 0.1
w = np.random.randn(D+1, 1)
target = generate_target_matrix(X,w,sigma)
print(target.shape)

(100, 1)


Q3 concept: pseudo inverse tackles the problem of computing inverse for least square problems when 
true inverse might not be defined i.e either X has depended variables or isn't sqauare matrix
method used to do so is single value decomposition which is what is used by analytical solver of numpy.linalg
package  ref: https://numpy.org/doc/stable/reference/generated/numpy.linalg.pinv.html 

Ideation : to analyse behavior wrt to scale and to plot log-log plots  vary the input size of N
in powers of 10s   source : self

Method to estimate time : use time library and mark time at the beginning and at end of computation 
and time duration would be given by start-end source : chat gpt

In [7]:
def time_computation(N,D =10):
    duration = []
    for n in N:
        ##use random matrix of size N to check computation
        t = np.random.randn(n)
        ## matrix having same size as that of input matrix
        X = np.random.randn(n,D)
        ##mark start time
        start = time.time()
        ##compute inv using np
        np.linalg.pinv(X) @ t
        ##mark end time
        end = time.time()
        ##compute duration of computation
        d = end - start
        ## to make a plot i need a list of duration against different values of N
        duration.append(d)
    return duration


In [13]:
## analyse 
##genearate N for analysis
N = [10**n for n in range(8)]  ##10 data points
##generate computation time
T = time_computation(N)
##plot logN vs logT
plt.figure(figsize=(10, 6))
plt.subplot(1,2,1)
plt.plot(np.log10(N), np.log10(T) , marker='o')
plt.title('Time Taken upto 2.5 x 10^7')
plt.xlabel('log of size of input (N)')
plt.ylabel('log of Time Taken (seconds)')
plt.grid(True)
##kernel crashed for N = 8 onwards so seperately printing 4 x 10^7(hit and trial) onward for analysis
N_high = [(35+n)* 10**6 for n in range(10)]
T_high = time_computation(N_high)
plt.subplot(1,2,2)
plt.plot(np.log10(N_high), np.log10(T_high) , marker='o')
plt.title('Time Taken post 2.5 x 10^7')
plt.xlabel('log of size of input (N)')
plt.ylabel('log of Time Taken (seconds)')
plt.grid(True)
plt.show()


: 

Observation :  for N upto 10^7  the computational time was within acceptable limits , the time taken was not even substantial until 10^7 even which was not more than 10s
However for even 5 ties 10^7 compututional ime had a sudden spike which caused the kernal to crash 

Reasoning : the method used by pinv of numpy package is SVD which involves computing three matrix for every matrix A of which inverse has to be computed and performing matrix multiplication since size of these matrices are NXD , NXN and DXD , no .of computations increases propotionally to N
Beyond a certain threshold, the system may run out of memory, leading to increased computational time or even a crash.
ADDITIONAL INPUT FROM CHAT GPT : total complexity of SVD is O(N.D^2)  this is due to combined effect of matrix mutliplication computation, transpose operations and inverse of singular values

